In [1]:
from getbibleverse import verse_lookup

import pandas as pd
import numpy as np

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [2]:
def store_verses(*passage):

    # create empty df to store verses
    verses_df = pd.DataFrame(columns=["bookname", "chapter", "verse", "text"])
    df = pd.DataFrame([])

    # create empty list to store input separately
    books = []
    chapters = []
    verses = []

    # break down input into separate lists
    for entry in passage:
        spaces = 0
        for i in range(0, len(entry)):
            if entry[i] == " ":
                spaces += 1
        
        if spaces == 1:
            book, ch_ver = entry.split(" ")
            chapter, verse_range = ch_ver.split(":")
        elif spaces == 2:
            book_num, book, ch_ver = entry.split(" ")
            book = book_num + " "  + book
            chapter, verse_range = ch_ver.split(":")
        else: 
            ValueError

        books.append(book)
        chapters.append(chapter)
        verses.append(verse_range)

    for i in range(len(books)):
        data = pd.json_normalize(verse_lookup(
            books[i], chapters[i], verses[i]))
        verses_df = pd.concat([verses_df, data], ignore_index=True)

    verses_df['book_chapter'] = verses_df['bookname'] + " " + verses_df['chapter']

    df['book_chapter'] = verses_df["book_chapter"].unique()
    df['verses'] =[i for i in verses]
    df['location'] = df['book_chapter'] + ":" + df['verses']

    verses_df = verses_df.groupby(['bookname', 'chapter'], sort=False).sum().reset_index()

    df['text'] = verses_df['text']

    text = "".join(df['text'])

    return text

In [3]:
store_verses("Proverbs 16:9", "Galatians 6:7-8", "Colossians 3:23", "Galatians 5:5", "Psalms 23:1")

'A person plans his course, but the Lord directs his steps. Do not be deceived. God will not be made a fool. For a person will reap what he sows, because the person who sows to his own flesh will reap corruption from the flesh, but the one who sows to the Spirit will reap eternal life from the Spirit. Whatever you are doing, work at it with enthusiasm, as to the Lord and not for people, For through the Spirit, by faith, we wait expectantly for the hope of righteousness. A psalm of David. The Lord is my shepherd, I lack nothing. '

In [9]:
def summary(per, *data):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(data[0])
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=' '.join(final_summary)
    return summary



"""
    nlp = spacy.load("en_core_web_sm")

    text = nlp(data[0])

    word_dict = {}
    for word in text:
        word = word.text.lower()
        if word in word_dict:
            word_dict[word] += 1
        else:
            word_dict[word] = 1

    sents = []
    sent_score = 0
    for index, sent in enumerate(text.sents):
        for word in sent:
            word = word.text.lower()
            sent_score += word_dict[word]
        sents.append((sent.text.replace("\n", " "), sent_score/len(sent), index)) 

    sents = sorted(sents, key=lambda x: -x[1])
    sents = sorted(sents[:sum_lgth], key=lambda x: x[2])

    summary_text = ""
    for sent in sents:
        summary_text += sent[0] + " "

    return summary_text"""

'\n    nlp = spacy.load("en_core_web_sm")\n\n    text = nlp(data[0])\n\n    word_dict = {}\n    for word in text:\n        word = word.text.lower()\n        if word in word_dict:\n            word_dict[word] += 1\n        else:\n            word_dict[word] = 1\n\n    sents = []\n    sent_score = 0\n    for index, sent in enumerate(text.sents):\n        for word in sent:\n            word = word.text.lower()\n            sent_score += word_dict[word]\n        sents.append((sent.text.replace("\n", " "), sent_score/len(sent), index)) \n\n    sents = sorted(sents, key=lambda x: -x[1])\n    sents = sorted(sents[:sum_lgth], key=lambda x: x[2])\n\n    summary_text = ""\n    for sent in sents:\n        summary_text += sent[0] + " "\n\n    return summary_text'

In [11]:
summary(0.65, store_verses("Proverbs 16:9", "Galatians 6:7-8", "Colossians 3:23", "Galatians 5:5", "Psalms 23:1"))

'For a person will reap what he sows, because the person who sows to his own flesh will reap corruption from the flesh, but the one who sows to the Spirit will reap eternal life from the Spirit. Whatever you are doing, work at it with enthusiasm, as to the Lord and not for people, For through the Spirit, by faith, we wait expectantly for the hope of righteousness. A person plans his course, but the Lord directs his steps. The Lord is my shepherd, I lack nothing.'